In [28]:
import numpy as np

import timeit

from LFM import SVDCustom

from surprise import Dataset, SVD, accuracy
from surprise.model_selection import train_test_split

In [83]:
seed = 17

# Датасет

URL: https://www.kaggle.com/datasets/prajitdatta/movielens-100k-dataset

Набор данны MovieLens был собран в рамках исследовательского проекта GroupLens
в Университете Миннесоты.

Этот набор данных включает в себя:
* 100 000 оценок (1-5) от 943 пользователей для 1682 фильмов.
* Каждый пользователь оценил не менее 20 фильмов.

Данные были собраны с помощью веб-сайта MovieLens (movielens.umn.edu) в течение семи месяцев с 19 сентября 1997 г.
по 22 апреля 1998 г, 1997 года по 22 апреля 1998 года. Эти данные были очищены - пользователи, у которых было менее 20 оценок или не было полной демографической информации, были удалены из этого набора данных.


In [84]:
data = Dataset.load_builtin('ml-100k')

In [85]:
data.raw_ratings[:20]

[('196', '242', 3.0, '881250949'),
 ('186', '302', 3.0, '891717742'),
 ('22', '377', 1.0, '878887116'),
 ('244', '51', 2.0, '880606923'),
 ('166', '346', 1.0, '886397596'),
 ('298', '474', 4.0, '884182806'),
 ('115', '265', 2.0, '881171488'),
 ('253', '465', 5.0, '891628467'),
 ('305', '451', 3.0, '886324817'),
 ('6', '86', 3.0, '883603013'),
 ('62', '257', 2.0, '879372434'),
 ('286', '1014', 5.0, '879781125'),
 ('200', '222', 5.0, '876042340'),
 ('210', '40', 3.0, '891035994'),
 ('224', '29', 3.0, '888104457'),
 ('303', '785', 3.0, '879485318'),
 ('122', '387', 5.0, '879270459'),
 ('194', '274', 2.0, '879539794'),
 ('291', '1042', 4.0, '874834944'),
 ('234', '1184', 2.0, '892079237')]

In [100]:
trainset, testset = train_test_split(data, test_size=0.25, random_state=seed)

In [101]:
print(trainset.n_users)
print(trainset.n_items)
print(trainset.n_ratings)

943
1627
75000


In [102]:
train_data = np.array([(int(trainset.to_raw_uid(u)),
                        int(trainset.to_raw_iid(i)),
                        int(r)) for (u, i, r) in trainset.all_ratings()])

test_data = np.array([(int(u), int(i), int(r)) for (u, i, r) in testset])

In [103]:
train_data[:10]

array([[ 254,  343,    2],
       [ 254,  214,    1],
       [ 254,  136,    4],
       [ 254,  496,    4],
       [ 254,  257,    3],
       [ 254,  167,    3],
       [ 254,  575,    3],
       [ 254, 1183,    4],
       [ 254,  951,    4],
       [ 254,   28,    4]])

Параметры обучения

In [104]:
n_factors = 10
n_epochs = 20
lr = 0.005
reg = 0.02

# Своя реализация

In [105]:
svd_custom = SVDCustom(
    n_factors=n_factors,
    n_epochs=n_epochs,
    lr=lr,
    reg=reg
)

In [111]:
%timeit svd_custom.fit(train_data)

1.17 s ± 22.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [93]:
rmse_custom, mae_custom = svd_custom.test(test_data)

In [109]:
print(round(rmse_custom,4))
print(round(mae_custom,4))

0.9724
0.7651


# Библиотечная реализация

In [95]:
svd_surprise = SVD(n_factors=n_factors,
                   n_epochs=n_epochs,
                   lr_all=lr,
                   reg_all=reg)

In [110]:
%timeit svd_surprise.fit(trainset)

95.3 ms ± 7.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [97]:
predictions = svd_surprise.test(testset)

In [98]:
predictions[:10]

[Prediction(uid='120', iid='405', r_ui=4.0, est=3.412809037610637, details={'was_impossible': False}),
 Prediction(uid='83', iid='254', r_ui=2.0, est=2.513058757723105, details={'was_impossible': False}),
 Prediction(uid='559', iid='311', r_ui=3.0, est=3.607584305419473, details={'was_impossible': False}),
 Prediction(uid='263', iid='921', r_ui=3.0, est=4.366432661628452, details={'was_impossible': False}),
 Prediction(uid='198', iid='871', r_ui=1.0, est=1.865979078990469, details={'was_impossible': False}),
 Prediction(uid='72', iid='515', r_ui=4.0, est=4.331550319280693, details={'was_impossible': False}),
 Prediction(uid='705', iid='252', r_ui=1.0, est=3.338251419048947, details={'was_impossible': False}),
 Prediction(uid='102', iid='840', r_ui=2.0, est=2.2123970798116517, details={'was_impossible': False}),
 Prediction(uid='417', iid='685', r_ui=1.0, est=3.062594870790964, details={'was_impossible': False}),
 Prediction(uid='497', iid='550', r_ui=4.0, est=3.257597234233468, details

In [99]:
surprise_rmse = accuracy.rmse(predictions)
surprise_mae = accuracy.mae(predictions)

RMSE: 0.9339
MAE:  0.7383
